# Rockets, rocket engines - Homework 1

### Imports

In [1]:
import numpy as np
import pandas as pd

### Load datas

In [2]:
stage_df = pd.read_csv('../data/import/data-1.csv')
display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s
0,1,458900,428300,2830
1,2,168300,157300,3140
2,3,50100,46600,3190


In [3]:
payload_df = pd.read_csv('../data/import/data-2.csv')
display(payload_df)
payload_mass = payload_df.loc[payload_df["component"] == "payload", "m_t"].values[0]
print(payload_mass)

,component,m_t
0,payload,22000


22000


## 1. How much Δ𝑣 accelaration can be achived with each stage?

In [4]:
def delta_v(we, m_0, m_f):
     return we * np.log(m_0 / m_f)

In [5]:
stage_df["m_struct_t"] = stage_df["m_0_t"] - stage_df["m_prop_t"]

In [6]:
display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t
0,1,458900,428300,2830,30600
1,2,168300,157300,3140,11000
2,3,50100,46600,3190,3500


In [7]:
upper_stages_masses = []
for i in range(len(stage_df)):
    upper_stages_masses.append(sum(stage_df.loc[i+1:, "m_0_t"]))

In [8]:
display(upper_stages_masses)

[218400, 50100, 0]

In [9]:
for i in range(len(stage_df)):
    stage_df.loc[i, "m_final_t"] = upper_stages_masses[i] + payload_mass + stage_df.loc[i, "m_struct_t"]

display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t,m_final_t
0,1,458900,428300,2830,30600,271000.0
1,2,168300,157300,3140,11000,83100.0
2,3,50100,46600,3190,3500,25500.0


In [10]:
for i in range(len(stage_df)):
    stage_df.loc[i, "delta_v"] = delta_v(stage_df.loc[i, "vel_out_m_s"], stage_df.loc[i, "m_0_t"], stage_df.loc[i, "m_final_t"])

display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t,m_final_t,delta_v
0,1,458900,428300,2830,30600,271000.0,1490.599206
1,2,168300,157300,3140,11000,83100.0,2215.908674
2,3,50100,46600,3190,3500,25500.0,2154.342753


In [11]:
display(stage_df['delta_v']) # m/s

0    1490.599206
1    2215.908674
2    2154.342753
Name: delta_v, dtype: float64

In [12]:
m_sum_prop_with_struct = stage_df["m_prop_t"].sum() + stage_df["m_struct_t"].sum()
print(f"Total mass with propellant {m_sum_prop_with_struct} kg") # kg
print(f"Total mass with propellant {m_sum_prop_with_struct / 1000} t") # t

Total mass with propellant 677300 kg
Total mass with propellant 677.3 t


## 2. Structural coefficient

In [13]:
def structural_coefficient(m_prop_param, m_struct_param):
    return m_struct_param / (m_struct_param + m_prop_param)

In [14]:
stage_df["struct_coeff"] = 0.0

In [15]:
for i in range(len(stage_df)):
    stage_df.loc[i, "struct_coeff"] = structural_coefficient(stage_df.loc[i, "m_prop_t"], stage_df.loc[i, "m_struct_t"])

In [16]:
display(stage_df[["stage", "struct_coeff"]])

,stage,struct_coeff
0,1,0.066681
1,2,0.065359
2,3,0.069860


## 3. Final velocity

In [17]:
delta_v_sum = stage_df["delta_v"].sum()
display(delta_v_sum)

np.float64(5860.8506337930385)

In [18]:
v_earth = 465 # m/s
deg = 45.95

In [19]:
v_final = delta_v_sum + v_earth * np.cos(np.radians(deg))

In [20]:
print(f"Final velocity: {v_final} m/s")
print(f"Final velocity: {v_final / 1000} km/s")

Final velocity: 6184.158553238991 m/s
Final velocity: 6.184158553238991 km/s


## 4. Height

In [21]:
g = 9.81
GM = 3.986e14 # m^3/s^2
R = 6.371e6 # m
print(v_final)

6184.158553238991


In [22]:
h = ((GM)/(v_final**2)) - R
print(f"Final height: {h} m")
print(f"Final height: {h / 1000} km")

Final height: 4051599.8121241033 m
Final height: 4051.599812124103 km


In [38]:
h = v_final**2 / (2 * g)
print(f"Final height: {h} m")
print(f"Final height: {h / 1000} km")

Final height: 1949226.1473801716 m
Final height: 1949.2261473801716 km


# Single stage javítás

In [23]:
w_s_stage = 3190.0 # m/s
z = np.exp(delta_v_sum / w_s_stage)
print(f"z: {z}")

z: 6.279292104874458


In [24]:
m0_single_stage = z * payload_mass
print(f"Single stage mass: {m0_single_stage} kg")
print(f"Single stage mass: {m0_single_stage / 1000} t")

# A háromfokozatú rakéta alapján vett szerkezeti : hajtóanyag arány
struct_to_prop_ratio_ref = stage_df["m_struct_t"].sum() / stage_df["m_prop_t"].sum()
print(f"Structural to propellant ratio: {struct_to_prop_ratio_ref}")

# Hajtóanyag tömeg (propellant) az arány alapján
m_prop_single_stage = (m0_single_stage -payload_mass) / (1 + struct_to_prop_ratio_ref)
print(f"Propellant mass: {m_prop_single_stage} kg")
print(f"Propellant mass: {m_prop_single_stage / 1000} t")

# Szerkezeti tömeg (strukturális)
m_struct_single_stage = m0_single_stage - payload_mass - m_prop_single_stage
print(f"Structural mass: {m_struct_single_stage} kg")
print(f"Structural mass: {m_struct_single_stage / 1000} t")

# 3 stage structural mass
m_struct_3_stage = stage_df["m_struct_t"].sum()
print(f"3 stage structural mass: {m_struct_3_stage} kg")
print(f"3 stage structural mass: {m_struct_3_stage / 1000} t")

# 3 stage and single stage ratio
ratio =  m_struct_single_stage / m_struct_3_stage
print(f"Ratio: {ratio}")

Single stage mass: 138144.42630723809 kg
Single stage mass: 138.1444263072381 t
Structural to propellant ratio: 0.0713381841189497
Propellant mass: 108410.6102339228 kg
Propellant mass: 108.4106102339228 t
Structural mass: 7733.816073315291 kg
Structural mass: 7.733816073315291 t
3 stage structural mass: 45100 kg
3 stage structural mass: 45.1 t
Ratio: 0.17148150938614837


induló tömeg

In [25]:
m_initial_single_stage = payload_mass * z
print(f"Initial mass of single stage: {m_initial_single_stage} kg")
print(f"Initial mass of single stage: {m_initial_single_stage / 1000} t")

Initial mass of single stage: 138144.42630723809 kg
Initial mass of single stage: 138.1444263072381 t


hajtóanyag tömeg

In [26]:
m_prop_single_stage = m_initial_single_stage * (1 - 1 / z)
print(f"Propellant mass of single stage: {m_prop_single_stage} kg")
print(f"Propellant mass of single stage: {m_prop_single_stage / 1000} t")

Propellant mass of single stage: 116144.42630723809 kg
Propellant mass of single stage: 116.14442630723809 t


## 5. Single stage old solution

In [27]:
w_s_stage = 3190.0 # m/s

In [28]:
print(f"total velocity: {delta_v_sum} m/s")

total velocity: 5860.8506337930385 m/s


In [29]:
z = np.exp(delta_v_sum / w_s_stage)
display(z)

np.float64(6.279292104874458)

### Initial mass

In [30]:
m_0_s_stage = m_sum_prop_with_struct * (z/(z-1))
print(f"Initial mass of the first stage: {m_0_s_stage} kg")
print(f"Initial mass of the first stage: {m_0_s_stage / 1000} t")

Initial mass of the first stage: 805593.7156242288 kg
Initial mass of the first stage: 805.5937156242288 t


### Structural mass

#### New structural mass

In [31]:
m_sum_prop = stage_df["m_prop_t"].sum()

In [32]:
print(f"Total propellant mass: {m_sum_prop} kg")
print(f"Total propellant mass: {m_sum_prop / 1000} t")

Total propellant mass: 632200 kg
Total propellant mass: 632.2 t


In [33]:
m_struct_s_stage = m_0_s_stage - payload_mass - m_sum_prop

In [34]:
print(f"Single stage structural mass: {m_struct_s_stage} kg")
print(f"Single stage Structural mass: {m_struct_s_stage / 1000} t")

Single stage structural mass: 151393.71562422882 kg
Single stage Structural mass: 151.39371562422883 t


In [35]:
# Sum of all structural masses in the 3 stage system
m_struct_sum = stage_df["m_struct_t"].sum()

In [36]:
print(f"Sum of all structural masses: {m_struct_sum} kg")
print(f"Sum of all structural masses: {m_struct_sum / 1000} t")

Sum of all structural masses: 45100 kg
Sum of all structural masses: 45.1 t


### Ratio

In [37]:
ratio =  m_struct_sum / m_struct_s_stage
print(f"Ratio of the structural masses: {ratio}")

Ratio of the structural masses: 0.29789875896792023
